## Convert all .tif files in a folder to image files (png/jpg)

In [44]:
import glob

from pprint import pprint

import rasterio
import matplotlib.pyplot as plt

import numpy as np

from skimage import exposure


from skimage.io import imsave, imread

In [45]:
# define parameters
output_path = '../../data/img_files_test/'

# define paths with raster data
rasters_file_path = '../../data/hilversum/*.tif'

# image specifics:
band_list = [3,2,1]
file_type = 'jpg' # jpg or png
upper_percentile = 98
lower_percentile = 2
max_single_value_count = 600


# misc
plot_results = True

In [46]:
# find files in raster folder
raster_files = glob.glob(rasters_file_path)

In [47]:
# check if the right files are selected
print(len(raster_files))
pprint(raster_files[0:2])

1
['../../data/hilversum/hilversum.tif']


In [59]:
# loop over all files and write them to output folder
for raster_file in raster_files[0:5]:


    dataset = rasterio.open(raster_file)
    
    # read and reformat raster data
    img = dataset.read()
    img_plot_raw = img[band_list,:,:]
    img_plot = np.rot90(np.fliplr(img_plot_raw.T))
    
    # correct exposure for each band individually
    img_plot_enhance = np.array(img_plot, copy=True)
    
    
    for band in range(3):
        
        # check max amount of a single value
        max_count_single_value = np.max(np.unique(img_plot, return_counts=True)[1])
        
        # if there are more than specific values set them as nan
        if max_count_single_value > max_single_value_count:
            no_data_value = img_plot.flatten()[np.argmax(np.unique(img_plot, return_counts=True)[1])]
            img_plot[img_plot == no_data_value] = np.nan 
            
        p_1, p_2 = np.nanpercentile(img_plot[:,:,band], (lower_percentile, upper_percentile))
        img_plot_enhance[:,:,band] = exposure.rescale_intensity(img_plot[:,:,band], 
                                                            in_range=(p_1, p_2), 
                                                            out_range = 'uint8')  
  

    
    # name string 
    orig_file_name = raster_file.split('/')[-1].split('.')[0]
    
    # write jpg file
    file_path_jpg = '{0}example_{1}.{2}'.format(output_path,orig_file_name, file_type)
    file_name_jpg = 'example_{0}.{1}'.format(orig_file_name, file_type)
    imsave(file_path_jpg, img_plot_enhance.astype('uint8'))
    
    
    if plot_results:
        plt.imshow(img_plot_enhance.astype(int))
        plt.title('Band used as [red, green, blue]: {}'.format(band_list))
        plt.show()
        
        print('--------------------------------------------')
        print('original file: \n', raster_file)
        print('--------------------------------------------')
        print('file written to: \n', file_path_jpg)
        print('--------------------------------------------')

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


--------------------------------------------
original file: 
 ../../data/hilversum/hilversum.tif
--------------------------------------------
file written to: 
 ../../data/img_files_test/example_hilversum.jpg
--------------------------------------------
